In [ ]:
from queue import Queue 
import threading
import itertools

In [ ]:
class Thurster:
    def __init__(self, seq, nb_amplifications=5, debug=False):
        self.seq = seq
        self.nb_amplifications = nb_amplifications
        self.amplification_queues = None
        self.create_queues()
        self.debug=debug
    
    def create_queues(self):
        self.amplification_queues = [Queue(2) for _ in range(self.nb_amplifications)]
        
    def googogo(self, phases):
        self._init_queue(phases)
        val = self._start_thread()
        self.create_queues()  # reset the queues after a run
        
        return val
    
    def _init_queue(self, phases):
        for index, phase in enumerate(phases):
            self.amplification_queues[index].put(phase)
        
        # The first input of Amplification circuit is 0
        self.amplification_queues[0].put(0)
        
    def _start_thread(self):
        threads = []
        # Create the threads
        for i in range(self.nb_amplifications):
            thread = threading.Thread(target=self._seq_reader, args=(i,"thread_{}".format(i)))
            threads.append(thread)
        
        # start the threads
        for thread in threads:
            thread.start()
        
        # wait for the threads to end
        for thread in threads:
            thread.join()
        
        # retrive the value
        return self.amplification_queues[0].get()
    
    def _next_amplification_queue(self, number):
        index = number if number < len(self.amplification_queues) else 0
        return self.amplification_queues[index]
        

    def _seq_reader(self, amplification, thread_id):

        def read_mod(mod_index):
            return mods[mod_index] if 0 <= mod_index < len(mods) else 0

        def get_val(index, mod_index):
            val = seq[index]

            return val if read_mod(mod_index) else seq[val]

        seq = self.seq.copy()
        go = True
        index = 0
        while(go):
            opcode = seq[index]
            val = opcode % 100
            mods = list(map(int, str(opcode)))[:-2]
            mods.reverse()

            if val in [1, 2, 7, 8]:
                op1 = get_val(index + 1, 0)
                op2 = get_val(index + 2, 1)
                if val == 1:
                    res = op1 + op2
                elif val == 2:
                    res = op1 * op2
                elif val == 7:
                    res = 1 if op1 < op2 else 0
                elif val == 8:
                    res = 1 if op1 == op2 else 0

                seq[seq[index + 3]] = res

                index += 4

            elif val == 3:
                # inut_val = int(input("input your value"))
                input_val = self.amplification_queues[amplification].get()
                
                if self.debug:
                    print("{} - input_val : {}".format(thread_id, input_val))
                    
                seq[seq[index + 1]] = input_val

                index += 2

            elif val == 4:
                output_val = get_val(index + 1, 0)
                
                if self.debug:
                    print("{} - output_val : {}".format(thread_id, output_val))
                
                self._next_amplification_queue(amplification + 1).put(output_val)
                index += 2

            elif val in [5, 6]:
                op1 = get_val(index + 1, 0)

                if val == 5 and op1 or val == 6 and not op1:
                    index = get_val(index + 2, 1)
                else:
                    index += 3

            elif val == 99:
                go = False
                continue

            else:
                raise ValueError("RIP")

In [ ]:
seq = [3,8,1001,8,10,8,105,1,0,0,21,34,59,76,101,114,195,276,357,438,99999,3,9,1001,9,4,9,1002,9,4,9,4,9,99,3,9,102,4,9,9,101,2,9,9,102,4,9,9,1001,9,3,9,102,2,9,9,4,9,99,3,9,101,4,9,9,102,5,9,9,101,5,9,9,4,9,99,3,9,102,2,9,9,1001,9,4,9,102,4,9,9,1001,9,4,9,1002,9,3,9,4,9,99,3,9,101,2,9,9,1002,9,3,9,4,9,99,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,99,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99]


thruster = Thurster(seq)
res = []
for combination in itertools.permutations([i for i in range(5)]):
    res.append(thruster.googogo(combination))

print(max(res))

In [ ]:
seq = [3,8,1001,8,10,8,105,1,0,0,21,34,59,76,101,114,195,276,357,438,99999,3,9,1001,9,4,9,1002,9,4,9,4,9,99,3,9,102,4,9,9,101,2,9,9,102,4,9,9,1001,9,3,9,102,2,9,9,4,9,99,3,9,101,4,9,9,102,5,9,9,101,5,9,9,4,9,99,3,9,102,2,9,9,1001,9,4,9,102,4,9,9,1001,9,4,9,1002,9,3,9,4,9,99,3,9,101,2,9,9,1002,9,3,9,4,9,99,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,99,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99]


thruster = Thurster(seq)
res = []
for combination in itertools.permutations([i + 5 for i in range(5)]):
    res.append(thruster.googogo(combination))

print(max(res))

In [ ]:
test_seq = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
thruster = Thurster(test_seq, debug=True)
thruster.googogo([4,3,2,1,0])

In [ ]:
test_seq = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
thruster = Thurster(test_seq, debug=True)
thruster.googogo([9,8,7,6,5])